<a href="https://colab.research.google.com/github/marieef/master-thesis/blob/main/BRAT_IAA_negation_KOPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from sklearn.metrics import cohen_kappa_score as kappa
from sklearn.metrics import f1_score as f1
from collections import defaultdict

In [3]:
# Du må ta en kopi av denne fila og endre så du får tilgang til ditt eget område hvis du vil bruke denne
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
ids1 = defaultdict(dict)
alle = os.listdir("/content/drive/MyDrive/Master/IAA_annotator1_gold_removed_Focus") #dette er mappa til annotør 1
for fname in alle:
    if fname[-4:] == ".txt":
        ids1[fname[:-4]]["text"] = fname
    elif fname[-4:] == ".ann":
        ids1[fname[:-4]]["ann"] = fname

ids2 = defaultdict(dict)
alle2 = os.listdir("/content/drive/MyDrive/Master/IAA_annotator2") #dette er mappa til annotør 2
for fname in alle2:
    if fname[-4:] == ".txt":
        ids2[fname[:-4]]["text"] = fname
    elif fname[-4:] == ".ann":
        ids2[fname[:-4]]["ann"] = fname


In [5]:
anno1 = defaultdict()
# Enn så lenge er anno2 lik anno1, må endre under.
anno2 = defaultdict()


def treatinfo(infostring):
    #takes the information and treats it, from Cues and Scopes
    span = []
    if ";" in infostring: #contains discontinuous spans
        semisplitta = infostring.split(";")
        first_split = semisplitta[0].split(" ")
        annotype = first_split[0]
        span.append(first_split[1:])
        for remaining in semisplitta[1:]:
            rem_split = remaining.split(" ")
            span.append(rem_split)
    else:
        splitta = infostring.split(" ")
        annotype = splitta[0]
        span.append(splitta[1:])
    #print("ann",{annotype:span})
    integer_span = []
    for x,y in span:
        x,y = int(x), int(y)
        integer_span.append([x,y])
    return {annotype:integer_span}


def getreps(annopath, textpath):
    #Splitter ikke siden det uansett bare er én setning på text
    with open(textpath,"r",encoding="utf-8") as indata:
        text = indata.read()
    with open(annopath,"r",encoding="utf-8") as indata:
        anno = indata.read().split("\n")
    infodict = defaultdict(list)
    negations = []
    for a in anno:
        splitta = a.split("\t")
        if a:
            if splitta[0][0] == "T": #entity
                info = treatinfo(splitta[1])
                strengen = splitta[2]
                kunstig_streng = " ".join([text[x:y] for t in info for x,y in info[t]])
                assert kunstig_streng == strengen #OK
                info["span"] = strengen
                infodict[splitta[0]] = info
    # we let all entities get added first
    for a in anno:
        splitta = a.split("\t")
        if a:
            if splitta[0][0] == "R": #negation relation
                info = splitta[1].split(" ")
                arg1 = info[1].split(":")[1]
                arg2 = info[2].split(":")[1]
                
                assert arg1 in infodict
                assert arg2 in infodict
                a1 = infodict[arg1]
                a2 = infodict[arg2]
                print("arg1", arg1, "infodict[arg1]", infodict[arg1])
                print("arg2", arg2, "infodict[arg2]", infodict[arg2])

                assert "Cue" in infodict[arg1]
                assert "Scope" in infodict[arg2]
                negation = {"Cue":[a1["Cue"],a1["span"]],"Scope":[a2["Scope"],a2["span"]]}
    
                negations.append(negation)
    #print(negations)
    return text, negations
    

print("\nANNOTATOR 1") 
for text_id in ids1:
    anno = ids1[text_id]["ann"]
    text = ids1[text_id]["text"]
    dirpath ="/content/drive/MyDrive/Master/IAA_annotator1_gold_removed_Focus"
    textoutput, negs = getreps(os.path.join(dirpath,anno),os.path.join(dirpath,text))
    anno1[text_id] = {"Negations":negs,"Text":textoutput}


#Bytt ut "ids1" med ids2 når den er klar.

print("\nANNOTATOR 2")
for text_id in ids2:
    anno = ids2[text_id]["ann"]
    text = ids2[text_id]["text"]
    dirpath ="/content/drive/MyDrive/Master/IAA_annotator2"
    textoutput, negs = getreps(os.path.join(dirpath,anno),os.path.join(dirpath,text))
    anno2[text_id] = {"Negations":negs,"Text":textoutput}

print(len(anno1),len(anno2))


ANNOTATOR 1
arg1 T1 infodict[arg1] {'Cue': [[25, 27]], 'span': 'in'}
arg2 T2 infodict[arg2] {'Scope': [[27, 36]], 'span': 'kontinens'}
arg1 T1 infodict[arg1] {'Cue': [[43, 53]], 'span': 'eksklusjon'}
arg2 T2 infodict[arg2] {'Scope': [[57, 82]], 'span': 'andre aktuelle tilstander'}
arg1 T1 infodict[arg1] {'Cue': [[142, 150]], 'span': 'opphevet'}
arg2 T2 infodict[arg2] {'Scope': [[151, 193]], 'span': 'evne til åpning av den anorektale vinkelen'}
arg1 T1 infodict[arg1] {'Cue': [[8, 12]], 'span': 'ikke'}
arg2 T2 infodict[arg2] {'Scope': [[0, 7], [13, 38]], 'span': 'Han var tidligere operert i buken'}
arg1 T1 infodict[arg1] {'Cue': [[125, 127]], 'span': 'in'}
arg2 T2 infodict[arg2] {'Scope': [[127, 136]], 'span': 'kontinens'}
arg1 T1 infodict[arg1] {'Cue': [[23, 24]], 'span': 'u'}
arg2 T2 infodict[arg2] {'Scope': [[24, 30]], 'span': 'ventet'}
arg1 T2 infodict[arg1] {'Cue': [[40, 49]], 'span': 'utelukkes'}
arg2 T1 infodict[arg2] {'Scope': [[5, 39], [50, 68]], 'span': 'lever , milt eller ind

In [62]:
def cue_iaa(annot1,annot2):
    # takes two dictionaries where the keys are texts and the values are lists of negations
    # exact match iaa
    a1 = []
    a2 = []
    for textid in annot1:
        t1 = annot1[textid]["Negations"]
        t2 = annot2[textid]["Negations"]
        a1cues = []
        a2cues = []
        for neg in t1:
            a1cues.append(str(neg["Cue"][0]))
        for neg in t2:
            a2cues.append(str(neg["Cue"][0]))
        # Siden det er umulig å vite akkurat hvilke ord som er cues, så lager vi en mengde, og så matcher opp mot
        # begge listene. Dette kan gi duplikater i tilfelle feil, men vil gi like mange duplikater for begge, så
        # det blir rettferdig 
        begge = set(a1cues + a2cues)
        
        #print(f"begge:\n{begge}")

        for index in begge:
            if index in a1cues:
                a1.append(1)
            else:
                a1.append(0)
            if index in a2cues:
                a2.append(1)
            else:
                a2.append(0)
    print("\nCUE a1:", a1)
    print("CUE a2:", a2)
    k_score = kappa(a1,a2)
    f1_score = f1(a1,a2)
    print("CUE kappa:", k_score)
    print("CUE F1:", f1_score)
        
cue_iaa(anno1,anno2)

# Exact scope matches - modified Petter's code. NB: Not used in thesis!!
def full_scope_iaa(annot1,annot2): 
    # takes two dictionaries where the keys are texts and the values are lists of negations
    # exact match iaa
    a1 = []
    a2 = []
    for textid in annot1:
        t1 = annot1[textid]["Negations"]
        t2 = annot2[textid]["Negations"]
        a1scopes = []
        a2scopes = []
        for neg in t1:
            a1scopes.append(str(neg["Scope"][0]))
        for neg in t2:
            a2scopes.append(str(neg["Scope"][0]))
     
        begge = set(a1scopes + a2scopes) # for å få noe annet enn 0 i kappa-score er vi avhengige av at ikke alle scopes finnes i a1 (og det er tilfelle)
        
        #print(f"begge:\n{begge}")

        for index in begge:
            if index in a1scopes:
                a1.append(1)
            else:
                a1.append(0)
            if index in a2scopes:
                a2.append(1)
            else:
                a2.append(0)
    print("\nFULL SCOPE a1:", a1)
    print("FULL SCOPE a2:", a2)
    k_score = kappa(a1,a2)
    f1_score = f1(a1,a2)
    print("FULL SCOPE kappa:", k_score)
    print("FULL SCOPE F1:", f1_score)

full_scope_iaa(anno1,anno2)

def overlaps(scope, word):
    #checks if a word overlaps with a scope
    #print(scope)
    wx,wy = word
    #print(wx,wy)
    for sx,sy in scope:
        #If a word starts or ends within the scope we count it as matching
        if sx <= wx <= sy or sx <= wy <= sy:
            return True
    return False

def gettokens(text):
    # takes a text and produces tokens with indices
    lista = []
    counter = 0
    for word in text.split(" "):
        y = counter + len(word)
        lista.append((word,(counter,y)))
        counter = y + 1
    return lista

def token_iaa(annot1,annot2):
    #token based IAA
    a1 = []
    a2 = []
    for textid in annot1:
        t = annot1[textid]["Text"]
        tokens = gettokens(t)
        #print("tokens", tokens) # tokens with index spans 
        # Just checking, this can actually be removed
        for pair in tokens: #can remove
            x,y = pair[1] #can remove
            assert pair[0] == t[x:y] #can remove
        negs = []
        for neg1 in annot1[textid]["Negations"]: # Lager par av alle kombinasjoner av negasjoner i en gitt setning 
            for neg2 in annot2[textid]["Negations"]:
                negs.append([neg1,neg2])
        #For each pair of possible negations
        for negpair in negs:
            n1 = negpair[0]
            n2 = negpair[1]
            if n1["Cue"] == n2["Cue"]: # den legger bare til i outputlistene hvis cuene er like (gir mening)
                for word_index in tokens:
                    if overlaps(n1["Scope"][0],word_index[1]):
                        a1.append(1)
                    else:
                        a1.append(0)
                    if overlaps(n2["Scope"][0],word_index[1]):
                        a2.append(1)
                    else:
                        a2.append(0)

    #usually when doing scope iaa, we assume that the cues are the same. We therefore get the same problem as with the cues
    # that we don't always know which negation corresponds to which, we therefore have to make pairs
    print("\nSCOPE a1:", a1)
    print("SCOPE a2:", a2)
    print(len(a1))
    print(len(a2))

    k_score = kappa(a1,a2)
    f1_score = f1(a1,a2)
    print("SCOPE TOKEN kappa:", k_score)
    print("SCOPE TOKEN F1:", f1_score)

token_iaa(anno1,anno2)



CUE a1: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
CUE a2: [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
CUE kappa: -0.04347826086956519
CUE F1: 0.9411764705882354

FULL SCOPE a1: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0]
FULL SCOPE a2: [1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1]
FULL SCOPE kappa: -0.29032258064516125
FULL SCOPE F1: 0.7058823529411764

SCOPE a1: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [63]:
# Added by me

def list_str_to_list(s): 
  #print("inp", s)
  #s = [int(x) for x in s[1:-1].split(", ")]
  s = s[1:-1].replace(",", "").split("] [")
  s = [[int(y) for y in x.replace("[", "").replace("]", "").split()] for x in s]
  #print("listr_str_to_list:", s)
  return s

# test = '[[25, 27], [1, 2]]' #-> [25 27|1 2]
# list_str_to_list(test)

def token_iaa_cues(annot1,annot2):
    #token based IAA
    a1 = []
    a2 = []
    for textid in annot1:
      t = annot1[textid]["Text"]
      tokens = gettokens(t)
      #print("tokens", tokens) # tokens with index spans 
      # Just checking, this can actually be removed
      t1 = annot1[textid]["Negations"]
      #print("t1", t1)
      t2 = annot2[textid]["Negations"]
      a1cues = []
      a2cues = []
      for neg in t1:
        a1cues.append(str(neg["Cue"][0]))
      for neg in t2:
        a2cues.append(str(neg["Cue"][0]))#

      # Siden det er umulig å vite akkurat hvilke ord som er cues, så lager vi en mengde, og så matcher opp mot
      # begge listene. Dette kan gi duplikater i tilfelle feil, men vil gi like mange duplikater for begge, så
      # det blir rettferdig 
      begge = set(a1cues + a2cues)
      #print(f"begge:\n{begge}")

      for cue in begge:
        if cue in a1cues:
            for word_index in tokens:
              as_list = list_str_to_list(cue)
              #print("as_list:", as_list)
              #print("word_index[1]:", word_index[1])
              if overlaps(as_list,word_index[1]):
                a1.append(1)
              else:
                a1.append(0)
        else:
            for word_index in tokens:
              a1.append(0)

        if cue in a2cues:
            for word_index in tokens:
              as_list = list_str_to_list(cue)
              #print("as_list:", as_list)
              #print("word_index[1]:", word_index[1])
              if overlaps(as_list,word_index[1]):
                a2.append(1)
              else:
                a2.append(0)
        else:
            for word_index in tokens:
              a2.append(0)

    print("\nCUE TOKEN a1:", a1)
    print("CUE TOKEN a2:", a2)
    print(len(a1))
    print(len(a2))

    k_score = kappa(a1,a2)
    f1_score = f1(a1,a2)
    print("CUE TOKEN kappa:", k_score)
    print("CUE TOKEN F1:", f1_score)

token_iaa_cues(anno1,anno2)


CUE TOKEN a1: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 